In [1]:
import os
import shutil  # shutil로 전체 디렉토리 삭제

# 파일 및 디렉토리 설정
persist_directory = "./chroma_db"  # Chroma DB가 저장될 디렉토리

try:
    # Chroma DB 초기화: 기존 디렉토리 삭제
    if os.path.exists(persist_directory):
        # 디렉토리와 그 안의 모든 내용 삭제
        shutil.rmtree(persist_directory)
        print(f"기존 Chroma DB 디렉토리 '{persist_directory}'이(가) 삭제되었습니다.")

    # 빈 디렉토리 생성 (선택 사항)
    os.makedirs(persist_directory, exist_ok=True)
    print(f"새로운 Chroma DB 디렉토리 '{persist_directory}'이(가) 생성되었습니다.")

except Exception as e:
    print(f"Chroma DB 초기화 중 오류 발생: {str(e)}")


기존 Chroma DB 디렉토리 './chroma_db'이(가) 삭제되었습니다.
새로운 Chroma DB 디렉토리 './chroma_db'이(가) 생성되었습니다.


In [4]:
import os
import json
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from dotenv import load_dotenv

load_dotenv()  # .env 파일 불러오기
OPENAI_KEY = os.getenv("OPENAI_KEY")

# 파일 및 디렉토리 설정
date_list_file = "date_list.txt"  # 날짜 목록 파일
data_dir = "summary"  # JSON 파일이 저장된 디렉토리
documents = []  # 문서 리스트 초기화

try:
    # 날짜 목록 읽기
    with open(date_list_file, "r", encoding="utf-8") as date_file:
        dates = [line.strip() for line in date_file if line.strip()]  # 빈 줄 제거

    # 날짜별 파일 처리
    for date_raw in dates:
        file_name = f"summary_{date_raw}.json"
        file_path = os.path.join(data_dir, file_name)

        if os.path.exists(file_path):  # 파일 존재 여부 확인
            # 날짜 변환: "250111" -> "2025-01-11"
            date = f"20{date_raw[:2]}-{date_raw[2:4]}-{date_raw[4:]}"

            # JSON 파일 로드
            with open(file_path, "r", encoding="utf-8") as f:
                summary_data = json.load(f)

                # 요약 내용을 문단별로 분리하여 문서로 추가
                if "summary_paragraphs" in summary_data:
                    paragraphs = summary_data["summary_paragraphs"]
                    for paragraph in paragraphs:
                        if paragraph.strip():  # 빈 문단 제외
                            # 문단에서 주제(1. 음식 등)를 추출
                            lines = paragraph.split("\n")
                            topic_line = lines[0].strip()
                            topic = topic_line.split(".")[1].strip() if "." in topic_line else "Unknown"

                            # 문단 내용만 추출 (주제 제외)
                            content = "\n".join(lines[1:]).strip()

                            # 문단별로 각각 Document 추가
                            documents.append(Document(
                                page_content=content,
                                metadata={"date": date, "topic": topic}
                            ))
        else:
            print(f"파일 '{file_name}'을(를) 찾을 수 없습니다.")

    print("데이터 통합 완료.")
    print(f"총 {len(documents)}개의 문서가 통합되었습니다.")

    # 통합된 문서 출력 (예시: 처음 5개 문서 출력)
    for doc in documents[:5]:
        print(f"문서 내용: {doc.page_content}")
        print(f"메타데이터: {doc.metadata}")

    # Step 2: 임베딩 및 벡터 저장소 생성
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)  # OpenAI Embeddings 사용

    # Chroma DB를 영구 저장하기 위한 디렉토리 설정
    persist_directory = "./chroma_db"  # 데이터베이스가 저장될 디렉토리

    # 변환된 리스트를 사용하여 벡터 저장소 생성
    vectorstore = Chroma.from_documents(
        documents,
        embedding=embeddings,
        persist_directory=persist_directory
    )

    # 벡터 저장소를 디스크에 저장
    vectorstore.persist()

    print("벡터 저장소 생성 및 영구 저장 완료!")

except FileNotFoundError as e:
    print(f"오류: {str(e)}")
except Exception as e:
    print(f"오류 발생: {str(e)}")

데이터 통합 완료.
총 8개의 문서가 통합되었습니다.
문서 내용: - 2025년 1월 11일, 사용자는 마라탕을 먹었다고 전했다. 토핑으로는 푸주를 선택했다. 또한, 딸기 탕후루를 먹었다고 언급했다.이어서 딸기가 들어간 빵을 사먹었다고 말했다. 그 빵에는 크림과 딸기 모카번이 들어있다고 말했다.
메타데이터: {'date': '2025-01-11', 'topic': '음식'}
문서 내용: - 2025년 1월 12일에는 다시 마라탕을 먹을 계획이라고 언급했다.
메타데이터: {'date': '2025-01-11', 'topic': '계획'}
문서 내용: - 사용자는 2025년 1월 12일에 매우 기분이 좋았고, 그냥 좋은 기분이어서 특별한 이유는 없다고 했다.
- 사용자는 맛있는 음식을 먹는 것을 좋아한다. 그날 딸기가 들어간 크림 모카번을 먹었는데, 커피 대신 빵과 함께 먹었다.
- 사용자는 소금빵도 좋아해하며, 마지막으로 먹은 것은 몇 주 전이었다. 그리고 학교 근처의 브레덴코에서 다음에 또 먹을 계획이 있다.
메타데이터: {'date': '2025-01-12', 'topic': '음식'}
문서 내용: - 사용자는 크림과 과일이 들어간 빵을 좋아하며, 치즈 빵은 별로 좋아하지 않는다. 과일 디저트 중에서는 케이크를 제일 좋아한다.
메타데이터: {'date': '2025-01-12', 'topic': '취향'}
문서 내용: - 사용자는 최근에 케이크를 먹은 것은 예수님의 생일 때였으며, 그 때 먹은 케이크는 생크림 케이크였다.
메타데이터: {'date': '2025-01-12', 'topic': '최근의 행동'}
벡터 저장소 생성 및 영구 저장 완료!


C:\Users\dasolkim7\AppData\Local\Temp\ipykernel_17376\1935560044.py:77: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [5]:
# 예시 쿼리
query = "선재 업고 튀어"
results = vectorstore.similarity_search(query, k=3)  # k=3: 상위 3개 결과

print("\n[유사도 검색 결과]")
for i, doc in enumerate(results):
    print(f"결과 {i+1}:")
    print(f"내용: {doc.page_content}")
    print(f"메타데이터: {doc.metadata}")
    print("-----------------------------")



[유사도 검색 결과]
결과 1:
내용: - 사용자는 2025년 1월 13일 점심에 '선재 업고 튀어라'라는 드라마를 보면서 엄마가 싸준 마라탕 도시락을 먹을 계획이라고 밝혔습니다. 사용자는 매운 음식을 아주 좋아하며, 특히 떡볶이와 짬뽕이 떠오르는 매운 음식이라고 언급하였습니다. 또한, 주말에도 마라탕을 먹을 계획이라고 말하였습니다.
메타데이터: {'date': '2025-01-13', 'topic': '음식'}
-----------------------------
결과 2:
내용: - 주말에는 교회에 다니는 것이 사용자의 계획 중 하나로, 교회에서 가장 좋아하는 활동은 예배드리기라고 언급하였습니다. 또한, 예배 후에는 친구들과 카페에 가기도 하며, 바스크 치즈케이크와 딸기 라떼를 주로 마신다고 말하였습니다.
메타데이터: {'date': '2025-01-13', 'topic': '주말 계획'}
-----------------------------
결과 3:
내용: - 2025년 1월 11일, 사용자는 마라탕을 먹었다고 전했다. 토핑으로는 푸주를 선택했다. 또한, 딸기 탕후루를 먹었다고 언급했다.이어서 딸기가 들어간 빵을 사먹었다고 말했다. 그 빵에는 크림과 딸기 모카번이 들어있다고 말했다.
메타데이터: {'date': '2025-01-11', 'topic': '음식'}
-----------------------------


In [6]:
# Chroma 벡터 스토어 객체가 vectorstore 변수에 있다고 가정
collection = vectorstore._collection

# include 파라미터에서 "ids"는 빼고, documents / embeddings / metadatas만 가져옴
stored_data = collection.get(include=["documents", "embeddings", "metadatas"])

print("=== 저장된 임베딩 정보 ===")

# documents, embeddings, metadatas는 리스트 형태이므로 각각 꺼내서 확인 가능
for i, (doc, embedding, metadata) in enumerate(zip(
    stored_data["documents"], 
    stored_data["embeddings"], 
    stored_data["metadatas"]
)):
    print(f"문서 번호: {i+1}")
    print(f"문서 내용: {doc}")
    print(f"메타데이터: {metadata}")    
    print(f"임베딩 벡터 길이: {len(embedding)}")
    # 임베딩 일부만 출력
    print(f"임베딩 앞부분: {embedding[:10]} ...\n")


=== 저장된 임베딩 정보 ===
문서 번호: 1
문서 내용: - 2025년 1월 11일, 사용자는 마라탕을 먹었다고 전했다. 토핑으로는 푸주를 선택했다. 또한, 딸기 탕후루를 먹었다고 언급했다.이어서 딸기가 들어간 빵을 사먹었다고 말했다. 그 빵에는 크림과 딸기 모카번이 들어있다고 말했다.
메타데이터: {'date': '2025-01-11', 'topic': '음식'}
임베딩 벡터 길이: 1536
임베딩 앞부분: [-0.01395782 -0.02383586  0.01116363 -0.01688319 -0.00661488 -0.00045709
 -0.02935864 -0.00451924 -0.01886404 -0.00846127] ...

문서 번호: 2
문서 내용: - 2025년 1월 12일에는 다시 마라탕을 먹을 계획이라고 언급했다.
메타데이터: {'date': '2025-01-11', 'topic': '계획'}
임베딩 벡터 길이: 1536
임베딩 앞부분: [-0.00614628 -0.02670565 -0.01150862 -0.01682276  0.00290607  0.00106749
 -0.03323426 -0.0047551  -0.02297868  0.00206911] ...

문서 번호: 3
문서 내용: - 사용자는 2025년 1월 12일에 매우 기분이 좋았고, 그냥 좋은 기분이어서 특별한 이유는 없다고 했다.
- 사용자는 맛있는 음식을 먹는 것을 좋아한다. 그날 딸기가 들어간 크림 모카번을 먹었는데, 커피 대신 빵과 함께 먹었다.
- 사용자는 소금빵도 좋아해하며, 마지막으로 먹은 것은 몇 주 전이었다. 그리고 학교 근처의 브레덴코에서 다음에 또 먹을 계획이 있다.
메타데이터: {'date': '2025-01-12', 'topic': '음식'}
임베딩 벡터 길이: 1536
임베딩 앞부분: [-0.0035786  -0.01289629  0.01612535 -0.03957436 -0.0061552  -0.00650141
 -0.02607221 -0.021